# Constrained Form Finding with Force Density - Example 1

In [1]:
from compas.colors import Color
from compas.datastructures import Mesh
from compas.geometry import Line, Point, Sphere, Vector
from compas_fd.solvers import fd_constrained_numpy
from compas_fd.constraints import Constraint
from compas_notebook.viewer import Viewer

## Base mesh

In [2]:
mesh = Mesh.from_meshgrid(dx=10, nx=10)

## FD Inputs

In [3]:
vertices = mesh.vertices_attributes("xyz")
fixed = list(mesh.vertices_where(vertex_degree=2))
edges = list(mesh.edges())
loads = [[0, 0, 0] for _ in range(len(vertices))]

# increase the force densities on the boundary edges

forcedensities = []
for edge in edges:
    q = 10.0 if mesh.is_edge_on_boundary(edge) else 1.0
    forcedensities.append(q)

# constraints

vertex_guid = {}
guid_constraint = {}

line = Line([0, 2, 0], [2, 10, 0])
constraint = Constraint(line)
guid = str(constraint.guid)
guid_constraint[guid] = constraint

vertex = list(mesh.vertices_where(x=0, y=10))[0]
vertex_guid[vertex] = guid

## Solve

In [4]:
constraints = [None] * len(vertices)
for vertex in vertex_guid:
    constraints[vertex] = guid_constraint[vertex_guid[vertex]]

result = fd_constrained_numpy(
    vertices=vertices,
    fixed=fixed,
    edges=edges,
    forcedensities=forcedensities,
    loads=loads,
    constraints=constraints,
)

## Update

In [5]:
for vertex, attr in mesh.vertices(data=True):
    attr["x"] = result.vertices[vertex, 0]
    attr["y"] = result.vertices[vertex, 1]
    attr["z"] = result.vertices[vertex, 2]

## Visualize

In [6]:
viewer = Viewer()
viewer.scene.clear()
viewer.scene.add(mesh, color=Color.grey())
viewer.show()

PyThreeJS SceneObjects registered.
